In [1]:
!pip install transformers
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping


     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 49.1 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
     |████████████████████████████████| 6.8 MB 24.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Using Titles

In [3]:
df = pd.read_json('/content/drive/MyDrive/fake_news_project/biased_news/tf_data/balanced_data_extremes.json')
df['bias'] += 1
df = df[['title', 'bias']]
train_set, test_set = train_test_split(df, test_size=0.20, random_state=42)

In [4]:
df['bias'].value_counts()

0    39787
1    39787
2    39787
Name: bias, dtype: int64

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
def biased_news_gen_title(df, tokenizer):
    def g():
        for row in df.itertuples():
            text = row.title
            label = row.bias
            tokenized = tokenizer(row.title, 
                                  max_length=32, 
                                  padding='max_length', 
                                  truncation=True)
            yield {k: np.array(tokenized[k]) for k in tokenized}, label
    return g

input_names = ['input_ids', 'token_type_ids', 'attention_mask']
data_types = ({k: tf.int32 for k in input_names}, tf.int64)
data_shapes = ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([]))

biased_news_train = tf.data.Dataset.from_generator(
    biased_news_gen_title(train_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(32).repeat(4)

biased_news_test = tf.data.Dataset.from_generator(
    biased_news_gen_title(test_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(32).repeat(4)

In [7]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model.compile(
                    optimizer=Adam(learning_rate=3e-5),
                    loss=SparseCategoricalCrossentropy(from_logits=True),
                    metrics='accuracy'
                  )

In [9]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=2,
                               restore_best_weights=True,
                               verbose=1)

In [10]:
history = model.fit(
                    biased_news_train,
                    validation_data=biased_news_test, 
                    epochs=15,
                    steps_per_epoch=128,
                    validation_steps=32,
                    callbacks=[early_stopping])

Epoch 1/15
128/128 [==============================] - 99s 568ms/step - loss: 0.9933 - accuracy: 0.4883 - val_loss: 0.8524 - val_accuracy: 0.6094
Epoch 2/15
128/128 [==============================] - 67s 527ms/step - loss: 0.8305 - accuracy: 0.6255 - val_loss: 0.7666 - val_accuracy: 0.6572
Epoch 3/15
128/128 [==============================] - 68s 528ms/step - loss: 0.7627 - accuracy: 0.6553 - val_loss: 0.7577 - val_accuracy: 0.6611
Epoch 4/15
128/128 [==============================] - 67s 527ms/step - loss: 0.7230 - accuracy: 0.6848 - val_loss: 0.6990 - val_accuracy: 0.6846
Epoch 5/15
128/128 [==============================] - 67s 527ms/step - loss: 0.7136 - accuracy: 0.6841 - val_loss: 0.6951 - val_accuracy: 0.7051
Epoch 6/15
128/128 [==============================] - 67s 526ms/step - loss: 0.6968 - accuracy: 0.6958 - val_loss: 0.6817 - val_accuracy: 0.7041
Epoch 7/15
128/128 [==============================] - 67s 526ms/step - loss: 0.6885 - accuracy: 0.7031 - val_loss: 0.6727 - val_ac

## Using Content

In [ ]:
df = pd.read_json('/content/drive/MyDrive/fake_news_project/biased_news/tf_data/balanced_data_extremes.json')
df['bias'] += 1
df = df[['content', 'bias']]
train_set, test_set = train_test_split(df, test_size=0.20, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
def biased_news_gen_content(df, tokenizer):
    def g():
        for row in df.itertuples():
            text = row.content
            label = row.bias
            tokenized = tokenizer(row.content, 
                                  max_length=512, 
                                  padding='max_length', 
                                  truncation=True)
            yield {k: np.array(tokenized[k]) for k in tokenized}, label
    return g

input_names = ['input_ids', 'token_type_ids', 'attention_mask']
data_types = ({k: tf.int32 for k in input_names}, tf.int64)
data_shapes = ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([]))

biased_news_train = tf.data.Dataset.from_generator(
    biased_news_gen_content(train_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(8).repeat(4)

biased_news_test = tf.data.Dataset.from_generator(
    biased_news_gen_content(test_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(8).repeat(4)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

In [ ]:
model.compile(
                    optimizer=Adam(learning_rate=3e-5),
                    loss=SparseCategoricalCrossentropy(from_logits=True),
                    metrics='accuracy'
                  )

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=2,
                               restore_best_weights=True,
                               verbose=1)

In [ ]:
history = model.fit(
                    biased_news_train,
                    validation_data=biased_news_test, 
                    epochs=15,
                    steps_per_epoch=128,
                    validation_steps=32,
                    callbacks=[early_stopping])

## Using both title and content

In [3]:
df = pd.read_json('/content/drive/MyDrive/fake_news_project/biased_news/tf_data/balanced_data_extremes.json')
df['bias'] += 1
df = df[['title', 'content', 'bias']]
train_set, test_set = train_test_split(df, test_size=0.20, random_state=42)

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
def biased_news_gen_title_content(df, tokenizer):
    def g():
        for row in df.itertuples():
            text = row.title + ' ' + row.content
            label = row.bias
            tokenized = tokenizer(text, 
                                  max_length=256, 
                                  padding='max_length', 
                                  truncation=True)
            yield {k: np.array(tokenized[k]) for k in tokenized}, label
    return g

input_names = ['input_ids', 'token_type_ids', 'attention_mask']
data_types = ({k: tf.int32 for k in input_names}, tf.int64)
data_shapes = ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([]))

biased_news_train = tf.data.Dataset.from_generator(
    biased_news_gen_title_content(train_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(16).repeat(4)

biased_news_test = tf.data.Dataset.from_generator(
    biased_news_gen_title_content(test_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(16).repeat(4)

In [6]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.compile(
                    optimizer=Adam(learning_rate=4e-5),
                    loss=SparseCategoricalCrossentropy(from_logits=True),
                    metrics='accuracy'
                  )

In [8]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=4,
                               restore_best_weights=True,
                               verbose=1)

In [9]:
history = model.fit(
                    biased_news_train,
                    validation_data=biased_news_test, 
                    epochs=30,
                    steps_per_epoch=128,
                    validation_steps=32,
                    callbacks=[early_stopping])

Epoch 1/30
128/128 [==============================] - 321s 2s/step - loss: 0.9463 - accuracy: 0.5034 - val_loss: 0.7176 - val_accuracy: 0.6797
Epoch 2/30
128/128 [==============================] - 292s 2s/step - loss: 0.7026 - accuracy: 0.6846 - val_loss: 0.6430 - val_accuracy: 0.7031
Epoch 3/30
128/128 [==============================] - 289s 2s/step - loss: 0.5992 - accuracy: 0.7339 - val_loss: 0.5180 - val_accuracy: 0.7832
Epoch 4/30
128/128 [==============================] - 289s 2s/step - loss: 0.5966 - accuracy: 0.7432 - val_loss: 0.5029 - val_accuracy: 0.7871
Epoch 5/30
128/128 [==============================] - 288s 2s/step - loss: 0.5218 - accuracy: 0.7754 - val_loss: 0.5245 - val_accuracy: 0.7617
Epoch 6/30
128/128 [==============================] - 288s 2s/step - loss: 0.4926 - accuracy: 0.7949 - val_loss: 0.4859 - val_accuracy: 0.7988
Epoch 7/30
128/128 [==============================] - 287s 2s/step - loss: 0.4579 - accuracy: 0.8057 - val_loss: 0.4513 - val_accuracy: 0.8203

In [12]:
model.evaluate(biased_news_test)

5972/5972 [==============================] - 5252s 879ms/step - loss: 0.3647 - accuracy: 0.8561


[0.36471977829933167, 0.8560717105865479]

In [14]:
model.save_pretrained('/content/drive/MyDrive/final_bias_news_model')

## Using left and right only

In [3]:
df = pd.read_json('/content/drive/MyDrive/fake_news_project/biased_news/tf_data/balanced_data_extremes.json')
df['bias'] += 1
df = df[['title', 'content', 'bias']]
df = df[df['bias'].isin([0, 2])]
df.loc[df['bias'] == 2, 'bias'] = 1
train_set, test_set = train_test_split(df, test_size=0.20, random_state=42)

In [4]:
df['bias'].value_counts()

0    39787
1    39787
Name: bias, dtype: int64

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [6]:
def biased_news_gen_title_content(df, tokenizer):
    def g():
        for row in df.itertuples():
            text = row.title + ' ' + row.content
            label = row.bias
            tokenized = tokenizer(text, 
                                  max_length=256, 
                                  padding='max_length', 
                                  truncation=True)
            yield {k: np.array(tokenized[k]) for k in tokenized}, label
    return g

input_names = ['input_ids', 'token_type_ids', 'attention_mask']
data_types = ({k: tf.int32 for k in input_names}, tf.int64)
data_shapes = ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([]))

biased_news_train = tf.data.Dataset.from_generator(
    biased_news_gen_title_content(train_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(16).repeat(4)

biased_news_test = tf.data.Dataset.from_generator(
    biased_news_gen_title_content(test_set, tokenizer),
    data_types, data_shapes
).shuffle(100).batch(16).repeat(4)

In [7]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model.compile(
                    optimizer=Adam(learning_rate=3e-5),
                    loss=SparseCategoricalCrossentropy(from_logits=True),
                    metrics='accuracy'
                  )

In [9]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=4,
                               restore_best_weights=True,
                               verbose=1)

In [10]:
 history = model.fit(
                    biased_news_train,
                    validation_data=biased_news_test, 
                    epochs=30,
                    steps_per_epoch=128,
                    validation_steps=32,
                    callbacks=[early_stopping])

Epoch 1/30
128/128 [==============================] - 315s 2s/step - loss: 0.6442 - accuracy: 0.6250 - val_loss: 0.6256 - val_accuracy: 0.6367
Epoch 2/30
128/128 [==============================] - 279s 2s/step - loss: 0.5472 - accuracy: 0.7124 - val_loss: 0.5372 - val_accuracy: 0.7500
Epoch 3/30
128/128 [==============================] - 279s 2s/step - loss: 0.4630 - accuracy: 0.7812 - val_loss: 0.4529 - val_accuracy: 0.8027
Epoch 4/30
128/128 [==============================] - 278s 2s/step - loss: 0.4256 - accuracy: 0.8003 - val_loss: 0.4124 - val_accuracy: 0.8320
Epoch 5/30
128/128 [==============================] - 280s 2s/step - loss: 0.3919 - accuracy: 0.8247 - val_loss: 0.4457 - val_accuracy: 0.8125
Epoch 6/30
128/128 [==============================] - 279s 2s/step - loss: 0.3692 - accuracy: 0.8335 - val_loss: 0.3979 - val_accuracy: 0.8164
Epoch 7/30
128/128 [==============================] - 278s 2s/step - loss: 0.3627 - accuracy: 0.8291 - val_loss: 0.4638 - val_accuracy: 0.7754